## End Goal : Submit if a given set of PassengerIds survived/not
```
PassengerId, Survived
123,0
234,1
...
```


### Data Dictionary

| **Variable** | **Definition**                             | **Key**                                        |
| ------------ | ------------------------------------------ | ---------------------------------------------- |
| survival     | Survival                                   | 0 = No, 1 = Yes                                |
| pclass       | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex          | Sex                                        |                                                |
| Age          | Age in years                               |                                                |
| sibsp        | # of siblings / spouses aboard the Titanic |                                                |
| parch        | # of parents / children aboard the Titanic |                                                |
| ticket       | Ticket number                              |                                                |
| fare         | Passenger fare                             |                                                |
| cabin        | Cabin number                               |                                                |
| embarked     | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |

### Variable Notes

**pclass**: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower
**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5
**sibsp**: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)
**parch**: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.


In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()

In [28]:
sns.set_style('darkgrid')

In [29]:
df = pd.read_csv('data/train.csv', index_col='PassengerId')
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Task 1 : Identify and Impune Missing Fields

In [30]:
df.info()
# Age has (891 - 714 = 177) missing columns
# Cabin has (891 - 204 = 687) missing columns

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [31]:
ser = df.isnull().sum()
ser = ser.loc[ser>0]
ser

Age         177
Cabin       687
Embarked      2
dtype: int64

In [32]:
df.loc[df.Embarked.isnull()]  # Fare is 80.0 for these 2 females carrying 1st class ticket. And Cabin is B28.

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [33]:
# Find female passengers whose  1st class Fare is 80.0 and if their cabin starts with B?
df.loc[(df.Sex=='female') & (df.Pclass==1) & (df.Fare==80.0000)] # Getting same 2 records with Embarked value asa nan.
df.loc[df.Fare==80.0000] # Getting same 2 records with Embarked value asa nan.
df.loc[df.Cabin=='B28'] # Getting same 2 records with Embarked value asa nan.
df.loc[df.Ticket=='113572'] # Getting same 2 records with Embarked value asa nan.

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [34]:
# Filling missing values with mode
mode = df.Embarked.mode().values[0]
# df[['Embarked']].fillna(mode, inplace=True) #Fails!
# df[['Embarked']] = df[['Embarked']].fillna(mode) #Works!
# df.loc[[62,830],'Embarked'] = [mode,mode] #Works!
df.fillna({'Embarked':mode}, inplace=True) # Works! This is nicer way. Use Dict {'coln_name': replacement_value}
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      0
dtype: int64

In [35]:
# With majority of Cabin columns having missing values, this column can be dropped
df.drop(columns=['Cabin'], inplace=True)
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Embarked      0
dtype: int64

In [36]:
# How to meaningfully impute Missing Ages???
df.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [37]:
''' This function is extracted to seperate file and can be seen imported after this snippet
def title(name):
    titles = ['Mr.','Mrs.','Miss.', 'Ms.', 'Master.', 'Major.', 'Mme.', 
              'Mlle.', 'Sir.', 'Lady.', 'Dr.', 'Rev.', 'Col.', 'Capt.', 'Don.', 'Dona.',
             'Countess.', 'Jonkheer.']
    for t in titles:
        if (t in name):
            return t
'''

from title import title
# df.loc[df.Name.apply(title).isnull()].Name
df['Title'] = df.Name.apply(title)
df.Title.value_counts()

Mr.          517
Miss.        182
Mrs.         125
Master.       40
Dr.            7
Rev.           6
Mlle.          2
Major.         2
Col.           2
Ms.            1
Capt.          1
Mme.           1
Jonkheer.      1
Countess.      1
Lady.          1
Sir.           1
Don.           1
Name: Title, dtype: int64

In [38]:
age_map = df.groupby(by=['Title']).mean().Age.round()
# age_map.to_pickle('pickle/age_map.pkl')
age_map.to_csv('data/age_map.csv', index=True, header=True)
age_map

Title
Capt.        70.0
Col.         58.0
Countess.    33.0
Don.         40.0
Dr.          42.0
Jonkheer.    38.0
Lady.        48.0
Major.       48.0
Master.       5.0
Miss.        22.0
Mlle.        24.0
Mme.         24.0
Mr.          32.0
Mrs.         36.0
Ms.          28.0
Rev.         43.0
Sir.         49.0
Name: Age, dtype: float64

In [39]:
# age_map['Col.']
# df[['Title','Age']].fillna(??)

''' This function is extracted to seperate file
def guess_age(row):
#     print(type(row)) # <class 'pandas.core.series.Series'?
    t = row['Title']
    row['Age'] = age_map[t]
    return row
'''
from guess_age import guess_age # Importing the extracted function above

# df.loc[df.Age.isnull()].apply(guess_age, axis=1)
df.loc[df.Age.isnull()] = df.loc[df.Age.isnull()].apply(guess_age, axis=1)
df.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
Title       0
dtype: int64

## Task 1 Completed Successfully!
* Added New Columns
  - 'Title'
* Imputed missing values:
  - Cabin : drop column because too many missing values
  - Age : Guess age by mean-age of the  group the person belongs to. See guess_age()
  - Embarked : Filled the gap with mode - 'S' in this case.

In [40]:
# Categorize Ages into  Groups -- Kids, Children, Adults, Aged
# ''' this method is extracted to seperate file
def age_group(age):
    if (age == np.nan): return np.nan
    elif (age<3): return 'baby'
    elif (age<12): return 'child'
    elif (age<20): return 'teen'
    elif (age<60): return 'adult'
    else: return 'aged'
# '''
# from age_group import age_group
# bins = [0,3,17,59,120]
# df['AgeGroup'] = pd.cut(df['Age'], bins)
# df['AgeGroup'].value_counts()
df['AgeGroup'] = df.Age.map(age_group)
df['AgeGroup']

PassengerId
1      adult
2      adult
3      adult
4      adult
5      adult
6      adult
7      adult
8       baby
9      adult
10      teen
11     child
12     adult
13     adult
14     adult
15      teen
16     adult
17      baby
18     adult
19     adult
20     adult
21     adult
22     adult
23      teen
24     adult
25     child
26     adult
27     adult
28      teen
29     adult
30     adult
       ...  
862    adult
863    adult
864    adult
865    adult
866    adult
867    adult
868    adult
869    adult
870    child
871    adult
872    adult
873    adult
874    adult
875    adult
876     teen
877    adult
878     teen
879    adult
880    adult
881    adult
882    adult
883    adult
884    adult
885    adult
886    adult
887    adult
888     teen
889    adult
890    adult
891    adult
Name: AgeGroup, Length: 891, dtype: object

In [41]:
# ax = sns.regplot(x="Age", y="Fare", data=df)
# df.loc[df.Age.isnull()==False].groupby(by=['Embarked', 'Pclass', 'Fare']).mean()
# df[['RoundedFare']] = df['Fare'].rou #apply(lambda x: (x//10)*10).value_counts()
''' Extracted to seperate file
def rounded_fare(fare):
    #Rounds fare to nearest 10
    return round( (fare/10) ) * 10
'''
from rounded_fare import rounded_fare
df['RoundedFare'] = df.Fare.map(rounded_fare) # (df.Fare /10).round() * 10 #.value_counts()
df.RoundedFare.value_counts()
tmp = df.loc[df.Age.isnull()==False].groupby(by=['Embarked', 'Pclass', 'AgeGroup']).mean()
# tmp.loc[['S',3,61]]
# queryExpr = 'Pclass in [1,3]' # Works! 
queryExpr = '(Embarked=="S") & (Pclass==3) & (AgeGroup=="aged")'
tmp.query(queryExpr)
# tmp.loc[[('S',3,61)]] # Works!
# df.drop(columns=['RoundedFare'], inplace=True)

,,,Survived,Age,SibSp,Parch,Fare,RoundedFare
Embarked,Pclass,AgeGroup,,,,,,
S,3,aged,0.333333,66.0,0.0,0.0,7.866667,10.0


In [42]:
# How to display just Fare and RoundedFare columns
# Ref.: https://jeffdelaney.me/blog/useful-snippets-in-pandas/
# Ref.: http://pandas.pydata.org/pandas-docs/stable/advanced.html#cross-section
fare_pivot_table = tmp.loc[:,['Fare','RoundedFare']] # Works to show all indices
fare_pivot_table.to_pickle('pickle/fare_pivot_table.pkl')

In [43]:
tmp.loc[('S',1,['baby','aged']),:] # Show selective indices and all features/columns

Survived    Age  SibSp     Parch        Fare  \
Embarked Pclass AgeGroup                                                 
S        1      aged          0.25  64.50   0.25  0.416667   59.998258   
                baby          0.50   1.46   1.00  2.000000  151.550000   

                          RoundedFare  
Embarked Pclass AgeGroup               
S        1      aged        60.833333  
                baby       150.000000

In [44]:
tmp.loc[('S',1,['baby','aged']),['Fare','RoundedFare']]

Fare  RoundedFare
Embarked Pclass AgeGroup                         
S        1      aged       59.998258    60.833333
                baby      151.550000   150.000000

## Task 2 : Identify Noise/Duplicate columns to be Dropped - Mark and Sweep
* Name
* Ticket
* Fare
* AgeGroup (Choosing Age feature for Modelling)
* Title

In [45]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title,AgeGroup,RoundedFare
PassengerId,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Mr.,adult,10
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Mrs.,adult,70
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss.,adult,10
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Mrs.,adult,50
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Mr.,adult,10


In [46]:
df.Ticket.value_counts()

CA. 2343             7
347082               7
1601                 7
347088               6
3101295              6
CA 2144              6
S.O.C. 14879         5
382652               5
349909               4
4133                 4
W./C. 6608           4
LINE                 4
113760               4
347077               4
113781               4
19950                4
17421                4
2666                 4
PC 17757             4
230080               3
PC 17755             3
13502                3
110413               3
345773               3
SC/Paris 2123        3
248727               3
C.A. 34651           3
239853               3
24160                3
PC 17582             3
                    ..
A/4. 20589           1
F.C. 12750           1
A/5. 2151            1
PC 17482             1
12460                1
345765               1
110564               1
345774               1
29011                1
2667                 1
SOTON/OQ 392086      1
A/4 45380            1
374746     

In [47]:
df.drop(columns=['Name','Ticket'], inplace=True) # Name, Ticket is just a meta-data and not a feature
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeGroup,RoundedFare
PassengerId,,,,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S,Mr.,adult,10
2,1,1,female,38.0,1,0,71.2833,C,Mrs.,adult,70
3,1,3,female,26.0,0,0,7.9250,S,Miss.,adult,10
4,1,1,female,35.0,1,0,53.1000,S,Mrs.,adult,50
5,0,3,male,35.0,0,0,8.0500,S,Mr.,adult,10


In [48]:
df.drop(columns=['Fare','AgeGroup','Title'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,RoundedFare
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,S,10
2,1,1,female,38.0,1,0,C,70
3,1,3,female,26.0,0,0,S,10
4,1,1,female,35.0,1,0,S,50
5,0,3,male,35.0,0,0,S,10


In [49]:
print(df.Sex.unique())
df.Sex.value_counts()

['male' 'female']


male      577
female    314
Name: Sex, dtype: int64

In [50]:
df['Embarked'].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [51]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked,RoundedFare
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,S,10
2,1,1,female,38.0,1,0,C,70
3,1,3,female,26.0,0,0,S,10
4,1,1,female,35.0,1,0,S,50
5,0,3,male,35.0,0,0,S,10


## Task 3 : Feature Engineering (on 2nd thoughts not doing it here and now!)
* Category Columns
  - Sex
  - Embarked
  - AgeGroup

In [52]:
# pd.get_dummies(data=df, drop_first=True)

## Final Task : Persist processed train-dataset

In [53]:
df.to_csv('data/train_processed_1.csv')

In [54]:
# df.head()

## Ancilarry EDA : Would Cabin column have helped, if not deleted???

In [55]:
# df.loc[df['Cabin'] == 'B96 B98']

## Digging a bit on the domain understanding
* References :https://www.rmg.co.uk/discover/explore/rms-titanic-fact-sheet
    - Passengers could be 1st, 2nd, 3rd class or Crew. How is a Crew member represented in the dataset??
* Is there any relevance of cabin-layout and survival? See diagram below: <img src='images/titanic.jpg'/>
    - 3rd class cabins are at the either end of the ship, bordered inside by 2nd class cabins. At the middle are 1st-class cabins.
    - For zoomable vectorized-picture of dec plan/layout see https://www.encyclopedia-titanica.org/titanic-deckplans/



In [56]:
# Split cabins as Cabin-Class, Cabin-Counts, Cabin-Start
'''
def split_cabins(df):
#     cdf = df['Cabin']
    for idx,row in df.iterrows():
        print(df['Cabin'])
        break;

split_cabins(df)        
'''

"\ndef split_cabins(df):\n#     cdf = df['Cabin']\n    for idx,row in df.iterrows():\n        print(df['Cabin'])\n        break;\n\nsplit_cabins(df)        \n"